In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
data = pd.read_csv('data/shakespeare-plays/Shakespeare_data.csv')
data[0:3]

,Dataline,Play,PlayerLinenumber,ActSceneLine,Player,PlayerLine
0,1,Henry IV,NaN,NaN,NaN,ACT I
1,2,Henry IV,NaN,NaN,NaN,SCENE I. London. The palace.
2,3,Henry IV,NaN,NaN,NaN,"Enter KING HENRY, LORD JOHN OF LANCASTER, the ..."


In [34]:
actLines = data['PlayerLine']
charPWord = actLines.str.len() #first hold total chars per line

wordPLine = actLines.str.split()
wordPLine = wordPLine.str.len()

charPWord = charPWord / wordPLine
charPWord
#actLines
#wordPLine
#unit_change = data[2:]
#unit_change.numdiff20162017 = pd.to_numeric(unit_change.numdiff20162017)
#unit_change
#data[100:200]

0         2.500000
1         5.600000
2         6.000000
3         4.222222
4         4.666667
5         6.571429
6         5.571429
7         5.125000
8         5.555556
9         6.000000
10        5.625000
11        6.166667
12        5.625000
13        5.125000
14        5.428571
15        5.833333
16        7.000000
17        4.444444
18        8.200000
19        5.500000
20        6.125000
21        4.625000
22        6.285714
23        5.428571
24        5.375000
25        5.750000
26        5.250000
27        6.000000
28        6.285714
29        5.428571
            ...   
111366    5.800000
111367    5.250000
111368    6.000000
111369    6.833333
111370    4.875000
111371    5.000000
111372    5.000000
111373    4.400000
111374    6.000000
111375    5.375000
111376    3.800000
111377    5.444444
111378    4.400000
111379    4.600000
111380    4.888889
111381    5.000000
111382    7.400000
111383    4.900000
111384    5.666667
111385    4.600000
111386    5.750000
111387    4.